In [36]:
import pandas as pd
import numpy as np

import time

from random import randint

import re
import string

import nltk


from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem.porter import *
stemmer = PorterStemmer()

from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.utils import shuffle

import json

In [6]:
df = pd.read_csv('./en_df_sample_30.csv')
print(df.shape)

(30, 7)


In [8]:
# df.head()

,index,song,year,artist,genre,lyrics,word_count
0,0,ego-remix,2009,beyonce-knowles,Pop,"Oh baby, how you doing? You know I'm gonna cut...",433.0
1,1,then-tell-me,2009,beyonce-knowles,Pop,"playin' everything so easy, it's like you seem...",258.0
2,2,honesty,2009,beyonce-knowles,Pop,If you search For tenderness It isn't hard to ...,170.0
3,3,you-are-my-rock,2009,beyonce-knowles,Pop,"Oh oh oh I, oh oh oh I [Verse 1:] If I wrote a...",522.0
4,4,black-culture,2009,beyonce-knowles,Pop,"Party the people, the people the party it's po...",312.0


# Replace Stopwords & Puncutation

In [54]:
def pre_represent(ly):
    ly_tokenized_words = word_tokenize(ly)
    filtered_stopwords_ly = list(filter(lambda word: word not in stopwords.words('english'), ly_tokenized_words)) # removing stopwords
    punctutaion_ly = [stemmer.stem(w) for w in filtered_stopwords_ly if not re.fullmatch('[' + string.punctuation + ']+', w)] # removing puncutation and then stemming
    return ' '.join(punctutaion_ly)

In [51]:
df['represent'] = df['lyrics'].apply(pre_represent)
df['word_count'] = df['represent'].str.split().str.len()
df = df[df['word_count'] >= 100]
df = df[df['word_count'] <= 1000]

In [52]:
corpus = df['represent'].tolist()
vectorizer = TfidfVectorizer(stop_words='english')
tfidf_matrix = vectorizer.fit_transform(corpus)
df_tfidf = pd.DataFrame(tfidf_matrix.toarray(), columns=vectorizer.get_feature_names())

In [53]:
df_tfidf['index'] = df_tfidf.index
df = df.merge(df_tfidf, on = 'index', how = 'left').head()
df.to_csv('./represented_lyrics.csv', index = False)

,index,song_x,year,artist,genre,lyrics,word_count,represent,12,1st,...,www,x8,ya,yard,ye,yeah,yo,young,yourl,youv
0,0,ego-remix,2009,beyonce-knowles,Pop,"Oh baby, how you doing? You know I'm gonna cut...",292,Oh babi you know I 'm gon na cut right chase s...,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,then-tell-me,2009,beyonce-knowles,Pop,"playin' everything so easy, it's like you seem...",129,playin everyth easi 's like seem sure still wa...,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,honesty,2009,beyonce-knowles,Pop,If you search For tenderness It isn't hard to ...,134,If search for tender It n't hard find you love...,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,you-are-my-rock,2009,beyonce-knowles,Pop,"Oh oh oh I, oh oh oh I [Verse 1:] If I wrote a...",355,Oh oh oh I oh oh oh I vers 1 If I wrote book s...,0.0,0.0,...,0.0,0.0,0.399787,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,black-culture,2009,beyonce-knowles,Pop,"Party the people, the people the party it's po...",211,parti peopl peopl parti 's pop sit around I se...,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
